**ESCUELA DE INGENIERÍA MECATRÓNICA**

## `PROCESAMIENTO DIGITAL DE SEÑALES E IMÁGENES`

### `Docente: Ms. Ing. Emerson Maximo Asto Rodriguez`

```
Práctica 10: Filtrado en el espacio
```

### 1. Implemente un algoritmo que permita realizar la correlacion 2D de una imagen de MxN con una mascara de mxn
* El algoritmo debe retornar una imagen de las mismas dimensiones que la imagen de entrada, recuerde que el "padding" usualmente se hace con ceros, no obstante se puede duplicar filas o columnas del borde, o hacerlo mediante un espejo sobre el borde.*

* El algoritmo tambien puede obtener la solución sin necesidad del "padding", agregando varias condicionales que permitan solo ponderar con los datos coincidentes.*

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [ ]:
def filtro_espacial(imagen, filtro):
    M, N = imagen.shape
    m, n = filtro.shape

    filas_add = (m - 1) // 2
    columnas_add = (n - 1) // 2

    img_pad = np.zeros((M + m - 1, N + n - 1), dtype=np.float32)
    img_pad[filas_add: filas_add + M, columnas_add: columnas_add + N] = imagen

    resultado = np.zeros_like(imagen)

    for i in np.arange(filas_add, filas_add + M):
        for j in np.arange(columnas_add, columnas_add + N):
            vecindad = img_pad[i-filas_add: i+filas_add+1, j-columnas_add: j+columnas_add+1]
            multiplicacion = vecindad * filtro
            resultado[i-filas_add, j-columnas_add] = np.sum(multiplicacion)

    return resultado

1. ¿Por qué es importante el uso de técnicas de padding en la aplicación de filtros espaciales? 
<p align="justify">
Debido a que el padding en filtros espaciales nos permiten procesar mejor los píxeles de los bordes de la imagen, ya que sin esto la imagen original empezaría a encogerse o desaparecerse mientras más filtros se vayan poniendo sin padding, es por ello que se utiliza este ante de aplicar el filtro y se asegura de que no se distorsione en los bordes  y que todos los bordes sean procesados, ya que se genera una ampliación de las dimensiones para poder utilizar el filtro sin preocupación de perder el tamaño original de la imagen.
<p>
2. Explica qué es un kernel (o máscara) en un filtro espacial y cómo influye su tamaño en el procesamiento de imágenes.
<p align="justify">
Un kernel o máscara es una pequeña matriz de números, en el que calculará (matemáticamente)el nuevo valor de un pixel a partir de los valores de sus pixeles vecinos, por lo que este kernel combina toda la información de los pixeles vecinos para obtener un nuevo valor en cada posición de la imagen.
Su tamaño influye en la intensidad y escala del efecto(al tener un kernel más pequeño solo se centra en los vecinos inmediatos, en cambio un kernel grande tiene un mayor rango de vecinos y así se obtiene un efecto más fuerte y pronunciado, lo que genera eliminación de detalles pequeños).
Además, influye en el costo computacional ya que su tamaño definirá la cantidad de cálculos que debe hacerse por cada píxel.
Por otro lado, también influye en la preservación de detalles, ya que tener un kernel pequeño asegura que se tenga mejor los detalles de alta frecuencia(texturas, bordes finos), en caso contrario, un kernel grande como se mencionó anteriormente, ocasiona que se agregue información sobre un área por lo que se borra o promedia los detalles de alta frecuencia, por lo que se prefiere si se requiere eliminar demasiado ruido.
<p>

In [ ]:
img = np.arange(100).reshape(10,10)
filtro = np.round(np.ones((5,7))/35, 3)

print(img)
print(filtro)

img_filtrada = filtro_espacial(img, filtro)
print("resultado \n", img_filtrada)

In [ ]:
img_ruido = cv2.imread('lab_images/ruido.png', 0)/255

filtro = np.ones((5,7))
filtro = filtro/np.sum(filtro)

img_ruido_filtrada = filtro_espacial(img_ruido, filtro)

plt.subplots(1,2, figsize=(15,20))
plt.subplot(121)
plt.imshow(img_ruido, cmap='gray')
plt.subplot(122)
plt.imshow(img_ruido_filtrada, cmap='gray', vmin=0, vmax=1)
plt.show()

### 2.- Implemente un algoritmo que permita mejorar solo los pixeles oscuros y de bajo contraste
*Puede modificar levemente el algoritmo realizado en el enunciado **1** para lograr el objetivo*

In [ ]:
def filtro_espacial_estadistico(imagen, selem = np.ones((3,3))):
    media_global = np.mean(imagen)
    desv_global = np.std(imagen)
    print("Media global:", media_global*0.3)
    print("Desviacion global:", desv_global)

    M, N = imagen.shape
    m, n = selem.shape

    filas_add = (m - 1) // 2
    columnas_add = (n - 1) // 2

    img_pad = np.zeros((M + m - 1, N + n - 1))
    img_pad[filas_add: filas_add + M, columnas_add: columnas_add+ N ] = imagen

    resultado = np.zeros_like(imagen)

    for i in np.arange(filas_add, filas_add + M):
        for j in np.arange(columnas_add, columnas_add + N):
            vecindad = img_pad[i-filas_add: i+filas_add+1, j-columnas_add: j+columnas_add+1]

            multiplicacion = vecindad * selem

            media_local = np.mean(multiplicacion)
            desv_local = np.std(multiplicacion)

            if (media_local<media_global) and (desv_local<0.8*desv_global) and (desv_local>0.2*desv_global):
                resultado[i-filas_add, j-columnas_add] = img_pad[i,j]*3
            else:
                resultado[i-filas_add, j-columnas_add] = img_pad[i,j]

    return resultado

1. ¿Cuál es el objetivo de este algoritmo y qué tipo de imágenes se benefician más de su aplicación?
<p align="justify">
Este algoritmo está relacionado con filtros espaciales estadísticos, lo cual permite analizar las características de las estadísticas locales (media y desviación estándar) de la imagen, para de esa manera modificarla selectivamente los pixeles, entonces se requiere destacar ciertas zonas de la imagen sin tener que perjudicar o alterar el contenido, en otras palabras, selecciona los pixeles que se deben modificar según las condiciones de la manera local y global.
Por otra parte, este filtro al querer alzar las imágenes con sombras u oscuras, entonces beneficias a aquellas que presentan demasiada oscuridad y no afecta al fondo, también aquellas que están presente a contraluz(una persona en una zona oscura y relativamente suave, le subirá el brillo sin modificar nada a lo que está a su alrededor ya que se considerará una media local alta), también es sobresaliente en imágenes con sombras profundas, entonces le subirá el brillo aquellas partes oscuras e incluso para las imágenes del sector salud o científico.
<p>

2. Vecindad y Kernel: ¿Cómo se define la vecindad de un píxel en este código y cuál es la importancia de la variable selem en este contexto?
<p align="justify">
La vecindad de un pixel se define por el tamaño del kernel o elemento estructurante (selem), en el cual por defecto se ha establecido en el código como uno de 3x3 ( np.ones((3,3)))
Además, en el código la variable que dice "vecindad", al presentar (i,j), se está generando una extracción de los valores vecinos centrados a ese pixel.
<p align="justify">

Entonces el rol que cumple la variable selem en este caso es para:
<p align="justify">

- Definir el alcance o tamaño espacial (m,n), en el que va a determinar los pixeles dentro de la vecindad que se van a considerar en el análisis.

-  Ponderación o la influencia en el cálculo de las estadísticas locales: Participa en la variable multiplicación para generar esta operación elemento por elemento de la vecindad y de esa forma también actúa en el peso de los cálculos de media y desviación local.
Así que si se modifica los valores de selem se genera otros cálculos, en el que si fuera de 2 en el centro y 1 en el borde, entonces el código daría más relevancia al pixel central y a sus vecinos inmediatos al calcular las estadísticas.
<p>
3. ¿Qué efecto tiene el factor de escala de 1.5 en los píxeles mejorados y cómo podría ajustarse para obtener diferentes resultados?
<p align="justify">
Cuando el pixel cumple con las condiciones estadísticas(media y desviación), su valor se multiplica por 1.5, lo que significa que ese pixel se vuelve más brillante, en el cual modifica sin tener que llegar a saturarlas o convertirlas en completamente blancas, pero en este caso cuando se hace un cambio a 3, trae como consecuencia que se genere un realce más fuerte y pueda ir produciendo mayor saturación o perder detalles.
<p align="justify">
Por lo cual, si se quiere hacer un ajuste para poder comprobar y analizar los resultados, se tendría que colocar un valor menor a 1.5 si se necesita un alzamiento de brillo más natural, manteniendo cierta parte de color oscura, en caso contrario se coloca un factor mayor a 1.5, lo que provocaría que el efecto sea más drástico o si se requiere que se tenga un fuerte brillo y contraste en las partes oscuras, pero teniendo en cuenta que mayor sea ese factor originaria que se pierda toda la información del pixel o de la zona oscura.
<p>


In [ ]:
emma = np.uint16(cv2.imread('./lab_images/ajedrez.jpg', 0))
emma_filtrada = filtro_espacial_estadistico(emma, selem = np.ones((5,5)))
plt.subplots(1,2, figsize=(20,20))
plt.subplot(121)
plt.imshow(emma, cmap='gray')
plt.subplot(122)
plt.imshow(emma_filtrada, cmap='gray', vmin=0, vmax=255)
plt.show()

### 3.- Implemente los filtros estadisticos min, max, moda y mediana


In [ ]:
from scipy.stats import mode

def filtro_espacial_estadistico(imagen, selem = np.ones((3,3)), filter_type='median'):
    M, N = imagen.shape
    m, n = selem.shape

    filas_add = (m - 1) // 2
    columnas_add = (n - 1) // 2

#     img_pad = np.zeros((M+ filas_adicionales*2, N + columnas_adicionales*2))
    img_pad = np.zeros((M + m - 1, N + n - 1))
    img_pad[filas_add: filas_add + M, columnas_add: columnas_add+ N ] = imagen

    resultado = np.zeros_like(imagen)

    for i in np.arange(filas_add, filas_add + M):
        for j in np.arange(columnas_add, columnas_add + N):
            vecindad = img_pad[i-filas_add: i+filas_add+1, j-columnas_add: j+columnas_add+1]

            multiplicacion = vecindad * selem

            if filter_type =='min':
                resultado[i-filas_add, j-columnas_add] = np.min(multiplicacion)
            elif filter_type == 'max':
                resultado[i-filas_add, j-columnas_add] = np.max(multiplicacion)
            elif filter_type == 'median':
                resultado[i-filas_add, j-columnas_add] = np.median(multiplicacion)
            elif filter_type == 'mode':
                resultado[i-filas_add, j-columnas_add] = mode(multiplicacion, axis=None, keepdims=False).mode

    return resultado

1. ¿Qué es un filtro estadístico en el procesamiento de imágenes y cómo se aplica en el código mostrado?

- Un filtro estadístico se encarga de procesar cada píxel reemplazando su valor por una estadística calculada sobre una vencidad local, como por ejemplo: mínimo, máximo, mediana o moda. Es una operación no lineal que usa información local para reducir ruido, realzar características o extraer valores.

- Lo que el código hace es calcular el tamaño de vencidad mxn del selem y crea una imagen con padding de ceros de tamaño (M+m-1,N+n-1). Copia la imagen original en el centro para poder extraer vecindades centradas en todos los pixeles sin salirse de rango. Realiza una extraccion para cada posicion central que es multiplicada por el selem que por defecto en la funcion se define como una matriz de 1 de 3x3, al multiplicar el selem actua como como pondereacion o mascara. Luego realiza un calculo estadistico depende al tipo de filtro que se va a realizar, cuyo resultado se escribe en la imagen de salida en la posicion.

2. ¿Qué diferencias existen entre los filtros de tipo min, max, median y mode? Explica en qué situaciones sería adecuado usar cada tipo.

- Filtro tipo min:
<p align="justify">
Este filtro se encarga de reemplazar el pixel por el valor minimo dentro de la vecindad, esto atenua zonas brillantes locales y tiende a oscurecer, ademas que elimina pequeños atributos brillantes. Es recomendado usar este filtro para poder eliminar "pepitas" brillantes asiladas, quitar salidas puntuales y además también es útil en la extracción de fondo en imágenes binarizadas.

- Filtro tipo max:
<p align="justify">
Este tipo de filtro se encarga de reemplazar el pixel por el valor máximo dentro de la vecindad, esto aclara zonas locales, elimina los puntos oscuros, es similar a una dialtación. Su uso es recomendado para poder eliminar "pepitas" oscuras aisladas, rellenar huecos pequeños y realzar píxeles brillantes dispersos. Se debe de tener cuidado en su uso porque puede expandir regiones brillantes y rellenar estructuras pequeñas.

- Filtro tipo median:
<p align="justify">
Este filtro se encarga de tomar la mediana de los valores de la vecindad, con esto se encarga de preservar los bordes mejor que un filtro promedio y de eliminar el ruido impulsivo eficazmente. Su uso es recomendado para poder quitar el ruido impulsivo, limpieza general donde no se quiere perder border, es el más usado para ruido impulsivo. Tiene una ventaja con respecto a los dos filtros anteriores porque no desplaza localmente la intensidad, sino que conserva los tonos.

- Filtro tipo mode:
<p align="justify">
Este filtro se encarga de tomar el valor que aparece con mayor frecuencia en la vecindad, esto tiende a reafirmar valores dominantes locales, lo cuál es útil en imágenes cuantizadas o con pocos niveles, además puede eliminar ruido cuando el ruido produce valores únicos y el fondo tiene un valor dominante. Es recomendado usarlo en imágenes con valores discretos o clasificaciones como mapas segmentados, imágenes binarias. Se debe de tener precaución en el uso de este filtro porque calcular la moda es más costoso computacionalmente y con imágenes con muchos valores distintos la moda puede ser poco representativa o inestable.
<p>

In [ ]:
#MINIMO
emma = np.uint16(cv2.imread('./lab_images/emma.jpg', 0))
emma_filtrada = filtro_espacial_estadistico(emma,filter_type='min')
plt.subplots(1,2, figsize=(15,20))
plt.subplot(121)
plt.imshow(emma, cmap='gray')
plt.subplot(122)
plt.imshow(emma_filtrada, cmap='gray')
plt.show()

In [ ]:
#MAX
emma = np.uint16(cv2.imread('./lab_images/emma.jpg',0))
emma_filtrada = filtro_espacial_estadistico(emma, filter_type='max')
plt.subplots(1,2, figsize=(15,20))
plt.subplot(121)
plt.imshow(emma, cmap='gray')
plt.subplot(122)
plt.imshow(emma_filtrada, cmap='gray')
plt.show()

In [ ]:
#MEDIANA
emma = np.uint16(cv2.imread('./lab_images/ruido.png', 0))
emma_filtrada = filtro_espacial_estadistico(emma,filter_type='median')
plt.subplots(1,2, figsize=(15,20))
plt.subplot(121)
plt.imshow(emma, cmap='gray')
plt.subplot(122)
plt.imshow(emma_filtrada, cmap='gray')
plt.show()

In [ ]:
#MODA
emma = np.uint16(cv2.imread('./lab_images/ruido.png', 0))
emma_filtrada = filtro_espacial_estadistico(emma, filter_type='mode')
plt.subplots(1,2, figsize=(15,20))
plt.subplot(121)
plt.imshow(emma, cmap='gray')
plt.subplot(122)
plt.imshow(emma_filtrada, cmap='gray')
plt.show()

### 4.- Implemente los filtros espaciales de suavizado y nitidez explicados en clase

Utilizando la funcion **convolve2d(img, kernel, mode= "same")** de **scipy.signal** y los kernels: 
(tambien se puede usar **correlate2d**)
* Promedio
* Gaussiano
* Laplaciano


In [ ]:
#PROMEDIO
from scipy import signal

img = cv2.imread("./lab_images/scarlett_oc.png", 0)/255
kernel = np.ones((11,11))/121

out = signal.convolve2d(img, kernel, mode= "same")

plt.figure(figsize=(15,15))
plt.subplot(121)
plt.imshow(img, cmap="gray")
plt.subplot(122)
plt.imshow(np.abs(out), cmap="gray", vmin=0, vmax=1)
plt.show()

In [ ]:
#GAUSSIANO
from scipy import signal

img = cv2.imread("./lab_images/scarlett_oc.png", 0)/255
# kernel = np.array([[1,2,1], [2,4,2], [1,2,1]])/16
kernel = np.array([1,4,7,4,1])*np.array([[1],[4],[7],[4],[1]])
kernel = kernel/np.sum(kernel)

out = signal.convolve2d(img, kernel, mode= "same")

plt.figure(figsize=(15,15))
plt.subplot(121)
plt.imshow(img, cmap="gray")
plt.subplot(122)
plt.imshow(np.abs(out), cmap="gray", vmin=0, vmax=1)
plt.show()

In [ ]:
#LAPLACIANO
from scipy import signal
A = 1.5
img = cv2.imread("./lab_images/scarlett_oc.png", 0)/255
kernel = np.array([[-1,-1,-1], [-1,A+8,-1], [-1,-1,-1]])

out = signal.convolve2d(img, kernel, mode= "same")

plt.figure(figsize=(15,15))
plt.subplot(121)
plt.imshow(img, cmap="gray")
plt.subplot(122)
plt.imshow(out, cmap="gray", vmin=0, vmax=1)
plt.show()

**Utilizando OpenCV:**
* cv2.blur(img, (Tamaño_kernel))
* cv2.GaussianBlur(img,(Tamaño_kernel),desviacion_estandar)
* cv2.medianBlur(img,Tamaño_kernel)
* cv2.bilateralFilter(img,tamaño_kernel,sigmacolor,sigmaspace)  [Mas info](https://homepages.inf.ed.ac.uk/rbf/CVonline/LOCAL_COPIES/MANDUCHI1/Bilateral_Filtering.html)

In [ ]:
img = cv2.imread("./lab_images/scarlett_oc.png")[..., ::-1]

out = cv2.blur(img, (7,7))

plt.figure(figsize=(15,15))
plt.subplot(121)
plt.imshow(img, cmap="gray")
plt.subplot(122)
plt.imshow(out, cmap="gray", vmin=0, vmax=1)
plt.show()

In [ ]:
out = cv2.GaussianBlur(img,(7,7), 2)

plt.figure(figsize=(15,15))
plt.subplot(121)
plt.imshow(img, cmap="gray")
plt.subplot(122)
plt.imshow(out, cmap="gray", vmin=0, vmax=1)
plt.show()

In [ ]:
img_joker = cv2.imread('./lab_images/ruido.png')[..., ::-1]

img_joker_filtrada = cv2.medianBlur(img_joker, 7)

plt.subplots(1,2, figsize=(15,20))
plt.subplot(121)
plt.imshow(img_joker, cmap='gray')
plt.subplot(122)
plt.imshow(img_joker_filtrada, cmap='gray')
plt.show()

In [ ]:
out = cv2.bilateralFilter(img,7, 160, 160)

plt.figure(figsize=(15,15))
plt.subplot(121)
plt.imshow(img, cmap="gray")
plt.subplot(122)
plt.imshow(out, cmap="gray", vmin=0, vmax=1)
plt.show()

1. ¿Cuál es la diferencia fundamental entre los filtros de suavizado (como el de promedio y el Gaussiano) y los filtros de nitidez (como el Laplaciano)?
<p align="justify">
- La principal diferencia es el objetivo que tiene cada uno de estos filtros, el de suavizado se encarga de eliminar detalla de alta frecuencia y producir una imagen más "suave" mientras que el filtro de nitidez tiende a realzar cambios locales en la intensidad, aumentando la percepción de detalle y contraste local. Otras diferencias son que el suavizado atenúa las frecuencias altas, reduce el ruido y su kernel está normalizado para mantener el nivel promedio de la imagen mientras que la nitidez realza o pasa las frecuencias altas, atenúa las frecuencias bajas y su kernel tiene suma igual a 0, por lo que detecta cambios sin aportar componente de baja frecuencia.
<p>

2. ¿Cuáles son las principales diferencias en los resultados obtenidos al aplicar un filtro de promedio versus un filtro Gaussiano?

- Peso del kernel: El filtro promedio tiene todos los coeficientes iguales y el píxel de salida es la media aritmética de la vecindad, mientras que el gaussiano, son pesos centrados mayores y decrecen suavemente haica afuera, donde el centro tiene más influencia.
- Respuesta de frecuencia: El promedio tiene una respuesta tipo sinc con oscilaciones mientras que la gaussiana tiene atenuación suave y monótona en frecuencias.
- Efecto visual: El filtro promedio tiene un blureo fuerte y homogéneo; suaviza texturas uniformemente, puede produci bordes más difusos y pérdida de contraste local, mientras que el gaussiano suaviza pero preserva mejor la estructura local, aplica una transición más natural.
3. ¿Qué propiedades tiene el kernel Laplaciano y cómo contribuye a la nitidez de la imagen? ¿Qué efectos visuales produce al aplicarse a una imagen?
- El kernel Laplaciano es un filtro de segunda derivada que mida la curvatura local de la intensidad, detecta cambios rápidos en intensidad en todas las direcciones, donde la forma típica es una matriz de 3x3 donde el centro es el 8 y su alrededor es el -1. Las propiedades matemática es que es isotrópico en la práctica, la suma de sus coeficientes es cero por eso actúa como filtro pasa-altas y resalta regiones con alta segunda derivada.
- Este filtro contribuye a la nitidex de la imagen, extrayendo las componentes de alta frecuencia, donde si se suma una versión escalada del mapa Laplaciano  la imagen original obtienes un realce de bordes, esto incrementa el contraste local en los bordes que genera una percepción con mayor nitidez.
- El efecto visual que produce es tener bordes más marcados, con contornos y texturas más visibles, aumenta la percepción de detalle en regiones con transiciones de intensidad. Lo negativo es que amplifica el ruido, crea halos alrededor de bordes si la ganacia es alta y produce un exceso de contraste en texturas finas si se aplica sin suavizar antes.
4. ¿Qué significa el argumento mode="same" en la función convolve2d y cómo afecta el tamaño de la imagen resultante después de aplicar el filtro?
- El modo same de dice a convolve2d que devuelva una salida del mismo tamaño que la imagen de entrada, internamente la convolución se calcula centrando el kernel sobre cada píxel de la imagen de entrada, para bordes toma en cuanta cómo se rellenan los valores fuera del límite.
- EL tamaño de la imagen resultante es el mismo que el de la imagen de entrada, mientras que los valores cercanos al vorde dependen del manejo de fronteras.

### 5.- Implemente un algoritmo para muestre el uso de la mascara de desenfoque (unsharp mask) y el filtro de altoaumento (Highboost)

In [ ]:
img = cv2.imread("./lab_images/scarlett_oc.png")[..., ::-1]/255

blur = cv2.blur(img, (7,7))

mask = img - blur

unsharp_image = mask + img

plt.figure(figsize=(20,15))
plt.subplot(131)
plt.imshow(img, cmap="gray")
plt.subplot(132)
plt.imshow(mask, cmap="gray")
plt.subplot(133)
plt.imshow(unsharp_image, cmap='gray', vmin=0, vmax=1)
plt.show()

1. ¿Cómo funciona la máscara de desenfoque (unsharp mask) para realzar los detalles de una imagen, y cuál es el papel del filtro de suavizado en este proceso?
---

La **máscara de desenfoque (Unsharp Mask)** realza detalles tomando la parte de alta frecuencia de una imagen.  
Primero se aplica un **suavizado (blur)** para obtener una versión sin bordes. Luego se calcula la **máscara** como la diferencia:

mask = Imag - I_blur

Esta máscara contiene los detalles y bordes. Después se suman esos detalles a la imagen original:

I_unsharp = Imag + mask

El **filtro de suavizado** es fundamental porque determina qué tanto detalle se elimina (bajas frecuencias) y, por lo tanto, qué tanto detalle se realza al volver a sumarlo.Tipificado en el codigo :

{blur = cv2.blur(img, (7, 7))}
En resumen este asignación aplica un filtro de suavizado (low-pass filter) con un kernel de 7×7, donde esto produce una imagen desenfocada, sin detalles ni bordes.


---

2.  ¿En qué se diferencia el filtro de alto aumento (Highboost) de la máscara de desenfoque estándar, y cómo se puede ajustar el factor de amplificación (boosting factor) para controlar el nivel de realce en la imagen?
---

El **filtro de alto aumento (Highboost)** es una versión generalizada de la máscara de desenfoque estándar.  
En la máscara de desenfoque tradicional, el realce se obtiene sumando la máscara de detalles una sola vez:

unsharp = Imag + (Imag - I_blur)

En el filtro Highboost, se añade un **factor de amplificación A (>1)** que permite aumentar la contribución de los detalles:

highboost = Imag + A * (Imag - I_blur)

La diferencia principal es que en **unsharp mask A = 1**, mientras que en **Highboost A > 1**, lo cual incrementa la intensidad del realce.  
Al aumentar el valor de A, los bordes y texturas se vuelven más visibles:  
- A cercano a 1 produce un realce suave.  
- A entre 1.5 y 3 genera mayor nitidez y contraste en los detalles.

Por lo tanto, el **boosting factor A** controla directamente el nivel de realce aplicado a la imagen.
Asi como se muestra en la siguiente compilación de codigo:


In [ ]:
img = cv2.imread("./lab_images/scarlett_oc.png")[..., ::-1]/255

blur = cv2.blur(img, (7,7))

mask = img - blur

unsharp_image = mask + img

# Highboost (A > 1)
A = 5 
highboost = img + A * mask


plt.figure(figsize=(20,15))

plt.subplot(141)
plt.title("Imagen Original")
plt.imshow(img)
plt.axis("off")

plt.subplot(142)
plt.title("Máscara (Detalles)")
plt.imshow(mask )  
plt.axis("off")

plt.subplot(143)
plt.title("Unsharp Mask por defecto donde (A=1)")
plt.imshow(unsharp_image)
plt.axis("off")

plt.subplot(144)
plt.title("Highboost (A=5)")
plt.imshow(highboost)
plt.axis("off")

plt.show()

### 6.- Implemente un algoritmo para mostrar la gradiente de una imagen
**Use las mascaras:**
* Roberts
* Prewit
* Sobel
* Scharr

In [ ]:
from scipy import signal
img = cv2.imread("./lab_images/frutas.jpg", 0) / 255.0

# Máscaras Roberts
kernelh = np.array([[0, -1],
                    [1,  0]])

kernelv = np.array([[-1, 0],
                    [ 0, 1]])
# Convolución
outh = signal.convolve2d(img, kernelh, mode="same")
outv = signal.convolve2d(img, kernelv, mode="same")
# Combinación
outf = outh + outv

# Mostrar resultados
plt.figure(figsize=(20,15))
plt.subplot(221)
plt.imshow(img, cmap="gray")
plt.title("Imagen Original")

plt.subplot(222)
plt.imshow(outh * 5, cmap="gray", vmin=0, vmax=1)
plt.title("Gradiente Horizontal (Roberts Gx)")

plt.subplot(223)
plt.imshow(outv * 5, cmap="gray", vmin=0, vmax=1)
plt.title("Gradiente Vertical (Roberts Gy)")

plt.subplot(224)
plt.imshow(outf + img, cmap="gray", vmin=0, vmax=1)
plt.title("Bordes Combinados (Roberts)")

plt.show()


In [ ]:
# Kernels Prewitt
kernelh = np.array([[-1, -1, -1],
                    [ 0,  0,  0],
                    [ 1,  1,  1]])

kernelv = np.array([[-1, 0, 1],
                    [-1, 0, 1],
                    [-1, 0, 1]])

Gx = signal.convolve2d(img, kernelh, mode="same")
Gy = signal.convolve2d(img, kernelv, mode="same")
G  = np.sqrt(Gx**2 + Gy**2)

plt.figure(figsize=(20,15))

plt.subplot(231)
plt.imshow(img, cmap="gray")
plt.title("Imagen Original")

plt.subplot(232)
plt.imshow(Gx, cmap="gray")
plt.title("Prewitt Gx")

plt.subplot(233)
plt.imshow(Gy, cmap="gray")
plt.title("Prewitt Gy")

plt.subplot(234)
plt.imshow(G, cmap="gray")
plt.title("Magnitud del Gradiente (Prewitt)")

plt.subplot(236)
plt.imshow(img + G, cmap="gray")
plt.title("Imagen + Gradiente (Prewitt)")

plt.show()



In [ ]:
# Kernels Sobel
kernelh = np.array([[-1, -2, -1],
                    [ 0,  0,  0],
                    [ 1,  2,  1]])

kernelv = np.array([[-1, 0, 1],
                    [-2, 0, 2],
                    [-1, 0, 1]])

Gx = signal.convolve2d(img, kernelh, mode="same")
Gy = signal.convolve2d(img, kernelv, mode="same")
G  = np.sqrt(Gx**2 + Gy**2)

plt.figure(figsize=(20,15))

plt.subplot(231)
plt.imshow(img, cmap="gray")
plt.title("Imagen Original")

plt.subplot(232)
plt.imshow(Gx, cmap="gray")
plt.title("Sobel Gx")

plt.subplot(233)
plt.imshow(Gy, cmap="gray")
plt.title("Sobel Gy")

plt.subplot(234)
plt.imshow(G, cmap="gray")
plt.title("Magnitud del Gradiente (Sobel)")

plt.subplot(236)
plt.imshow(img + G, cmap="gray")
plt.title("Imagen + Gradiente (Sobel)")

plt.show()



In [ ]:
# Kernels Scharr
kernelh = np.array([[-3, -10, -3],
                    [ 0,   0,  0],
                    [ 3,  10,  3]])

kernelv = np.array([[-3,  0,  3],
                    [-10, 0, 10],
                    [-3,  0,  3]])

Gx = signal.convolve2d(img, kernelh, mode="same")
Gy = signal.convolve2d(img, kernelv, mode="same")
G  = np.sqrt(Gx**2 + Gy**2)

plt.figure(figsize=(20,15))

plt.subplot(231)
plt.imshow(img, cmap="gray")
plt.title("Imagen Original")

plt.subplot(232)
plt.imshow(Gx, cmap="gray")
plt.title("Scharr Gx")

plt.subplot(233)
plt.imshow(Gy, cmap="gray")
plt.title("Scharr Gy")

plt.subplot(234)
plt.imshow(G, cmap="gray")
plt.title("Magnitud del Gradiente (Scharr)")

plt.subplot(236)
plt.imshow(img + G, cmap="gray")
plt.title("Imagen + Gradiente (Scharr)")

plt.show()



Luego de implementar las mascaras de gradientes

### 1. ¿Cuál es la característica distintiva de la máscara de Roberts y por qué es menos precisa?

La máscara de **Roberts** se distingue porque calcula el gradiente usando **kernels muy pequeños de 2×2**, y detecta bordes principalmente en **diagonales**.  
Debido a este tamaño reducido, captura variaciones muy locales y se vuelve **más sensible al ruido** y **menos precisa en imágenes con gradientes suaves o detalles complejos**.  
Su simplicidad lo hace rápido, pero menos robusto que operadores de 3×3 como Prewitt, Sobel o Scharr.

---

### 2. ¿Cómo afectan Sobel y Scharr a la precisión en la detección de bordes y cuándo se elige cada uno?

Las máscaras de **Sobel** y **Scharr** usan kernels de 3×3 que suavizan en la dirección perpendicular al borde, lo que mejora la **estabilidad** y reduce el **ruido**.  
Sobel aplica pesos moderados en la dirección del gradiente, mientras que **Scharr** utiliza coeficientes optimizados que producen una estimación más **precisa y rotacionalmente simétrica** del gradiente.

- **Sobel** se prefiere cuando se busca un detector estable, económico y suficientemente preciso.  
- **Scharr** se elige cuando se requiere **mayor exactitud**, especialmente en aplicaciones que necesitan gradientes más fieles (visión por computadora, mediciones, análisis fino de contornos).

En resumen, **Scharr ofrece mayor precisión**, pero Sobel es más ligero computacionalmente.
